- Standard blackjack rules implemented 
- Displays cards 
- Old player record is kept in a pickle file
- Recognizes returning players based on names

                                     [Ashwin Lokkur]


In [1]:
class TextColor:
    color = {"RED":'\x1b[1;1;31;10m', "GREEN":'\x1b[1;1;32;10m', "END":'\x1b[0m',"BLUE":'\x1b[1;1;34;10m'}

In [2]:
class Player:
    __name = None
    __balance = None
    __cards = None
    __cards_value = None
    __ace_present = None
    __recent_bet = None
    def __init__(self, name, balance):
        self.__name = name
        self.__balance = balance
        self.__cards = []
    
    def __str__(self):
        return self.__name + " has a balance  of  " + str(self.__balance)
    
    def bet(self, money):
        if money <= self.__balance:
            self.__balance -= money
            self.__recent_bet = money
            return True
        elif self.__balance == 0:
            return "ZERO"
        else:
            return False
    
    def won(self, money):
         self.__balance += money

    def gets_card(self, cards):
        for c in cards:
            self.__cards.append(c)
            if c.get_rank() == "Ace":
                self.__ace_present = True
    
    # def display_cards(self):
    #     for c in self.__cards:
    #         print(c)

    def get_balance(self):
        return self.__balance
    
    def get_recent_bet(self):
        return self.__recent_bet
    
    def calc_card_value(self):
        self.__cards_value = [0]
        for c in self.__cards:
            self.__cards_value[0] += c.get_value()
        
        if self.__ace_present:
            self.__cards_value = [self.__cards_value[0], self.__cards_value[0] - 10]

    def get_card_value(self):
        return self.__cards_value

    def remove_value(self, n):
        self.__cards_value.remove(n)
    
    def new_hand(self):
        self.__cards = []
        self.__ace_present = False
        self.__recent_bet = None
        
    def get_cards(self):
        return self.__cards
    
    def get_name(self):
        return self.__name
    
    def visualize_cards(self):
        lines = [[],[],[],[],[],[],[]]
        for c in self.__cards:
            rank = c.get_rank()
            suit_symbol = c.get_suit_symbol()
            space = ' '
            if rank == 10:
                space = ''
            elif type(rank) is str:
                rank = rank[0]
            lines[0].append('┌─────────┐')
            lines[1].append('│{}{}       │'.format(rank, space))
            lines[2].append('│         │')
            lines[3].append('│    {}    │'.format(suit_symbol))
            lines[4].append('│         │')
            lines[5].append('│       {}{}│'.format(space, rank))
            lines[6].append('└─────────┘')
        return lines
    
    def display_cards(self):
        lines = self.visualize_cards()    
        for i in lines:
            for j in i:
                print(j, end='')
            print()

In [3]:
class Dealer(Player):

    __hidden_cards = []
    __balance = 0
    __cards_value_with_hidden = None
    __hidden_ace_present = None
    def __init__(self):
        Player.__init__(self, "Dealer", 0)    
    
    def gets_hole_card(self, card):
        for c in card:
            self.__hidden_cards.append(c)
            if c.get_rank() == "Ace":
                self.__hidden_ace_present = True
    
    def bet(self):
        pass

    def calc_card_value(self):
        Player.calc_card_value(self)
        self.__cards_value_with_hidden = Player.get_card_value(self)
        
        for c in self.__hidden_cards:
            if len(self.__cards_value_with_hidden) == 1:
                self.__cards_value_with_hidden[0] += c.get_value()
            elif len(self.__cards_value_with_hidden) >= 2:
                for v in range(len(self.__cards_value_with_hidden)):
                    self.__cards_value_with_hidden[v] += c.get_value()
        temp = []
        if self.__hidden_ace_present:
            for v in self.__cards_value_with_hidden:
                temp.append(v)
                temp.append(v-10)
    
    def get_card_value(self):
        return self.__cards_value_with_hidden

    def new_hand(self):
        Player.new_hand(self)
        self.__hidden_cards = []
        self.__hidden_ace_present = False
    
    def display_cards(self, display_hole_cards):
        lines = [[],[],[],[],[],[],[]]
        for c in self.__hidden_cards:
            if not display_hole_cards:
                lines = [['┌─────────┐'], ['│░░░░░░░░░│'], ['│░░░░░░░░░│'], ['│░░░░░░░░░│'], ['│░░░░░░░░░│'], ['│░░░░░░░░░│'], ['└─────────┘']]

            else:
                rank = c.get_rank()
                suit_symbol = c.get_suit_symbol()
                space = ' '
                if rank == 10:
                    space = ''
                elif type(rank) is str:
                    rank = rank[0]
                lines[0].append('┌─────────┐')
                lines[1].append('│{}{}       │'.format(rank, space))
                lines[2].append('│         │')
                lines[3].append('│    {}    │'.format(suit_symbol))
                lines[4].append('│         │')
                lines[5].append('│       {}{}│'.format(space, rank))
                lines[6].append('└─────────┘')
            
            temp = Player.visualize_cards(self)
            for i in range(len(temp)):
                # lines[i].append(temp[i])
                for j in temp[i]:
                    lines[i].append(j)
            for i in lines:
                for j in i:
                    print(j, end='')
                print()


In [4]:
import random
import itertools
class Deck:

    __suits = ("Diamonds", "Spades", "Clubs", "Hearts")
    __ranks = (2, 3, 4, 5, 6, 7, 8, 9, 10, "Jack", "Queen", "King", "Ace")
    __values = {2:2, 3:3, 4:4, 5:5, 6:6, 7:7,8:8, 9:9, 10:10, "Jack":10, "Queen":10, "King":10, "Ace":11}
    __cards = []
    __cards_in_deck = 0
    
    def __init__(self):
        self.__cards = list(itertools.product(self.__suits,self.__ranks))
        self.__cards_in_deck = len(self.__cards)
        for c in range(self.__cards_in_deck):
            cardObj = Card(self.__cards[c][0], self.__cards[c][1], self.__values[self.__cards[c][1]]) 
            self.__cards[c] = cardObj
        
    def shuffle(self):
        random.shuffle(self.__cards)

    def deal(self, n):
        ret = []
        for i in range(0, n):
            ret.append(self.__cards.pop())
        return ret

    def __str__(self):
        ret = ""
        for c in self.__cards:
            ret = ret + str(c) + "\n"
        return ret     
    
    def reset(self):
        self.__init__()
    

In [5]:
class Card:
    __rank = None
    __suit = None
    __value = None

    __suit_symbol = {"Spades" : '♠', "Diamonds" : '♦', "Hearts" : '♥', "Clubs" : '♣'}

    def __init__(self, suit, rank, value):
        self.__rank = rank
        self.__suit = suit
        self.__value = value

    def get_rank(self):
        return self.__rank 
    
    def get_suit(self):
        return self.__suit
    
    def get_value(self):
        return self.__value
    
    def set_value(self, value):
        self.__value = value
    
    def get_suit_symbol(self):
        return self.__suit_symbol[self.__suit]
    
    def __str__(self):
        return str(self.__rank) + " of " + self.__suit

In [47]:
import pickle 
from IPython.display import clear_output

FILE_NAME = "players.dat"

players = []
deck = None
player = None
dealer = None

def get_old_players():
    global players
    try:
        infile = open(FILE_NAME, "rb+")
        l = ''
        try:
            l = pickle.load(infile)
        except EOFError:
            players = []
            return
        while True:
            try:
                players.append(l)
                l = pickle.load(infile)
            except EOFError:
                break
    except FileNotFoundError:
        pass
def end_game(new_player):
    for i in players:
        if i['name'] == player.get_name():
            i['balance'] = player.get_balance()
    outfile = open(FILE_NAME, "wb")
    for p in players:
        pickle.dump(p, outfile)
    outfile.close()
    
def search_player(name):
    for p in players:
        if name == p['name']:
            return p
    return None

def start_round():
    global player
    global dealer

    player.new_hand()
    dealer.new_hand()

    player.gets_card(deck.deal(2))
    dealer.gets_hole_card(deck.deal(1))
    dealer.gets_card(deck.deal(1))

def display_table(dealer_hole_display = False):
    clear_output()
    print("---------------------------------------------")
    print("\t\tPlayer Cards")
    print("---------------------------------------------")
    # player.display_cards()
    player.display_cards()
    print("\n---------------------------------------------")
    print("\t\tDealer Cards")
    print("---------------------------------------------")
    # dealer.display_cards(dealer_hole_display)
    dealer.display_cards(dealer_hole_display)

def player_turn():
    global player
    display_table()

    while(True):
        player.calc_card_value()

        if len(player.get_card_value()) == 2 and player.get_card_value()[0] > 21:
            player.remove_value(player.get_card_value()[0])
        elif len(player.get_card_value()) == 2 and player.get_card_value()[1] > 21:
            player.remove_value(player.get_card_value()[1])

        if len(player.get_card_value()) == 1 and player.get_card_value()[0] > 21:
            return ("BUST", player.get_card_value())
        elif 21 in player.get_card_value():
            return ("BLACKJACK", player.get_card_value())
        
        ch = input("Hit or Stand? Press H or S: ")

        if ch.upper() == 'H':
            player.gets_card(deck.deal(1))
            player.calc_card_value()
        elif ch.upper() == 'S':
            return ("STAND", player.get_card_value())
        else:
            print("Invalid input. Please retry")
            continue
            
        display_table()
        
    return None

def dealer_turn():
    global dealer
    display_table(dealer_hole_display = True)
    while(True):
        dealer.calc_card_value()
        
        if len(dealer.get_card_value()) > 1 :
            for c in dealer.get_card_value():
                if c > 21:
                    dealer.remove_value(c)
        
        if len(dealer.get_card_value()) == 1 and dealer.get_card_value()[0] > 21:
            return("BUST",dealer.get_card_value())
        elif len(dealer.get_card_value()) == 1 and dealer.get_card_value()[0] < 17:
            pass
        elif len(dealer.get_card_value()) == 1 and dealer.get_card_value()[0] >= 17 and dealer.get_card_value()[0] < 21 :
            return ("STAND", dealer.get_card_value())
        elif 21 in dealer.get_card_value():
            return ("BLACKJACK", dealer.get_card_value())
        elif dealer.get_card_value() > player.get_card_value():
            return ("STAND", dealer.get_card_value())
        

        dealer.gets_card(deck.deal(1))
        dealer.calc_card_value()
        
        display_table(dealer_hole_display = True)
        
def check_win(p, d):
    
    if p[0] == "BUST":
        print(TextColor.color['RED']+"Player Busted!"+TextColor.color['END'])
        print(TextColor.color['RED']+"Player lost $"+str(player.get_recent_bet())+TextColor.color['END'])
    elif p[0] == "BLACKJACK":
        print(TextColor.color['GREEN']+"Player hit Blackjack!"+TextColor.color['END'])
        print(TextColor.color['GREEN']+"Player won $"+str(2*player.get_recent_bet())+TextColor.color['END'])
        player.won(2*player.get_recent_bet())
    elif d[0] == "BUST":
        print(TextColor.color['RED']+"Dealer Busted!"+TextColor.color['END'])
        print(TextColor.color['GREEN']+"Player won $"+str(2*player.get_recent_bet())+TextColor.color['END'])
        player.won(2*player.get_recent_bet())
    elif d[0] == "BLACKJACK":
        print(TextColor.color['GREEN']+"Dealer hit Blackjack!"+TextColor.color['END'])
        print(TextColor.color['RED']+"Player lost $"+str(player.get_recent_bet())+TextColor.color['END'])
    if p[0] == "STAND" and d[0] == "STAND":
        if p[1] > d[1]:
            print(TextColor.color['GREEN']+"Player won the round!"+TextColor.color['END'])
            print(TextColor.color['GREEN']+"Player won $"+str(2*player.get_recent_bet())+TextColor.color['END'])
            player.won(2*player.get_recent_bet())
        elif p[1] < d[1]:
            print(TextColor.color['RED']+"Player lost the round!"+TextColor.color['END'])
            print(TextColor.color['RED']+"Player lost $"+str(player.get_recent_bet())+TextColor.color['END'])
        else:
            print(TextColor.color['BLUE']+"It was a Draw!"+TextColor.color['END'])
            player.won(player.get_recent_bet())

def start_game():
    global deck
    global player
    global dealer
    deck = Deck()
    while(True):
        deck.shuffle()
        try:
            start_round()
        except IndexError:
            deck.reset()
            continue
        while(True):
            try:
                print(TextColor.color['BLUE']+player.get_name()+"'s balance is: $"+str(player.get_balance())+TextColor.color['END'])
                money = 0
                if player.get_balance() == 0:
                    can_play = False
                    print("Player has $0 balance. ")
                    m = int(input(TextColor.color['BLUE']+"How much would you like to buy-in for? "+TextColor.color['END']))
                    player.won(m)
                    continue
                elif player.get_balance() != 0:
                    money = int(input(TextColor.color['BLUE']+"How much do you want to bet? "+TextColor.color['END']))
                    if money == 0 or money > player.get_balance():
                        raise ValueError 
                    player.bet(money)
                    break
            except ValueError:
                print("Enter a valid input")
        
        p = player_turn()
        d = None
        if p[0] != "BUST" and p[0] != "BLACKJACK":
            d = dealer_turn()
        print("D",dealer.get_card_value())
        print("P",player.get_card_value())
        check_win(p, d)    
        ch = None
        while(True):
            ch = input("Do you want to continue? [Y/N]")
            
            if ch.upper() == 'Y':
                ch = True
                break
            elif ch.upper() == 'N':
                ch = False
                break
            else:
                print("Enter a valid choice.")
        if ch:
            continue
        else:
            break
def main():
    global player
    global dealer

    name = input("Enter your name: ")
    get_old_players()
    player_found = search_player(name)

    player = None
    new_player = False
    
    if player_found is not None:
        print(TextColor.color['BLUE']+"Welcome back, "+name+TextColor.color['END'])
#         print("You have a balance of "+str(player_found['balance']))
        player = Player(player_found['name'],player_found['balance'])
    else:
        temp = {}
        b=0
        while True:
            try:
                b = int(input("How much do you want to buy-in for? "))
            except ValueError:
                print("Please enter a valid number.")
            else:
                break
        temp['name'] = name
        temp['balance'] = b
        players.append(temp)
        player = Player(name,b)
        new_player = True
    
    dealer = Dealer()

    start_game()   
    end_game(new_player)
    print(TextColor.color['BLUE']+"Thank you playing with Casino 83, "+name+"."+TextColor.color['END'])
    
#     write_new_players(name, player.get_balance())
main()

---------------------------------------------
		Player Cards
---------------------------------------------
┌─────────┐┌─────────┐
│9        ││9        │
│         ││         │
│    ♥    ││    ♣    │
│         ││         │
│        9││        9│
└─────────┘└─────────┘

---------------------------------------------
		Dealer Cards
---------------------------------------------
┌─────────┐┌─────────┐┌─────────┐
│7        ││6        ││Q        │
│         ││         ││         │
│    ♠    ││    ♦    ││    ♦    │
│         ││         ││         │
│        7││        6││        Q│
└─────────┘└─────────┘└─────────┘
D [23]
P [18]
Dealer Busted!
Player won $200
Do you want to continue? [Y/N]n
Thank you playing with Casino 83, Ash.
